# PaperTrade

This notebook applies the generated Buy/Sell signals to DepotManager.

1. Notebook loads `MACD_BuySell_<par1>_<par2>_<par3>.ipynb`
1. Use papermill module to extract the data file `MACD_BuySell_<stock data file>_<par1>_<par2>_<par3>.csv` from the notebook.
1. Load the csv data
1. Run buy/sell data against DepotManager

In [43]:
# notebook name, where the 
nb_file = '../data/interim/MACD_BuySell_12_26_9.ipynb'

# initial balance of the depot 
balance = 1000


#### Some installs and imports

In [2]:
# Install a pip module in the current Jupyter kernel
import sys

try:   
    import seaborn as sns
except ImportError as err:
    print("Handling run-time error: ", err)
    print("Will now install missing module.")
    !{sys.executable} -m pip install seaborn
    
try:
    import papermill as pm
except ImportError as err:
    print("Handling run-time error: ", err)
    print("Will now install missing module.")
    !{sys.executable} -m pip install papermill


Handling run-time error:  No module named 'seaborn'
Will now install missing module.
    100% |################################| 215kB 2.0MB/s ta 0:00:01
Handling run-time error:  No module named 'papermill'
Will now install missing module.
    100% |################################| 51kB 1.0MB/s ta 0:00:01
    100% |################################| 133kB 1.2MB/s ta 0:00:01
    100% |################################| 61kB 2.3MB/s ta 0:00:01
    100% |################################| 4.6MB 727kB/s ta 0:00:01    13% |####                            | 645kB 916kB/s eta 0:00:05


    100% |################################| 552kB 1.8MB/s ta 0:00:01


In [81]:
# other imports
import os
import math

# manipulating data
import pandas as pd
import numpy as np
#from stockstats import StockDataFrame as Sdf

# plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.style as style 

# notebook parameterizing and automation 
import papermill as pm

## Load Buy/Sell Signals from a Notebook

Use papermill to load the notebook `nb_file`and extract the csv file location containing buy/sell signals. Finally, it loads the csv file into a dataframe. 

In [40]:
# load the notebook and extract the csv file location
nb = pm.read_notebook(nb_file)

# extract csv file location 
output_data_path = nb.dataframe[nb.dataframe['name']=='output_data_path']['value'].values[0]
csv = nb.dataframe[nb.dataframe['name']=='csv']['value'].values[0]
csv_file = os.path.join(output_data_path, csv )

In [41]:
stock = pd.read_csv(csv_file, delimiter=';', header=0)
stock.drop(['Unnamed: 0'], axis=1, inplace=True)

## Run DepotManager

Simulate the depot using the stock data and the buy/sell signals. 

In [67]:
module_path = os.path.abspath(os.path.join(os.getcwd(),'../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import DepotManager as dp

In [83]:
# instanciate the object
dpMngr = dp.DepotManager()
dpWKN = dpMngr.builtDepot('A2G8XR')
# charge up with balance
dpWKN.broker.balance = balance

In [84]:
stock.sort_index(ascending=True)

stock['balance'] = math.nan
stock.at[0, 'balance']=dpWKN.broker.balance

for i in range(1, len(stock)):

    avail_balance = dpWKN.broker.balance - 30
    avail_shares = dpWKN.totalStockinDepot()
    price = stock.at[i, 'close']
    date = stock.at[i, 'date']

    
    # buy
    if stock.at[i, 'signal'] == 1:
        print('buy')
        shares = math.floor(avail_balance/price)
        # dpWKN.buy(date, shares, price )
        
    # sell
    if stock.at[i, 'signal'] == -1:
        print('sell')
        # dpWKN.sell(date, avail_shares, price )
        
    # sell
    if stock.at[i, 'signal'] == 0:
        print('hold')
        # nothing to do here
    
    # Store the final balance in stock frame
    stock.at[i, 'balance'] = dpWKN.broker.balance

hold
sell
buy
hold
sell
hold
hold
hold
hold
hold
hold
hold
hold
buy
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
sell
hold
buy
hold
hold
hold
hold
hold
hold
hold
sell
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
buy
hold
sell
hold
hold
hold
hold
hold
hold
hold
hold
buy
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
sell
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
buy
hold
hold
hold
hold
hold
sell
hold
hold
hold
buy
hold
hold
hold
sell
buy
hold
hold
hold
hold
hold
sell
hold
hold
hold
hold
hold
hold
buy
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
sell
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold
hold

In [85]:
stock

,date,open,high,low,close,shares,volume,close_12_ema,close_26_ema,macd,macds,macdh,signal,balance
0,2017-08-21,174.3375,175.6815,173.1374,174.0495,776936,135380672,174.049500,174.049500,0.000000,0.000000,0.000000,0,1000.0
1,2017-08-22,175.2015,176.1615,174.8655,175.5855,714906,125526592,174.881500,174.847038,0.034462,0.019145,0.015316,0,1000.0
2,2017-08-23,175.5375,176.0655,174.0014,174.3375,800080,139733784,174.669177,174.663966,0.005210,0.013434,-0.008224,-1,1000.0
3,2017-08-24,174.4815,176.5935,174.4815,175.4415,725802,127510400,174.912969,174.881331,0.031638,0.019601,0.012037,1,1000.0
4,2017-08-25,175.3455,176.3535,174.8175,174.9135,673465,118082856,174.913113,174.888791,0.024322,0.021005,0.003317,0,1000.0
5,2017-08-28,173.8574,175.2495,173.7614,174.0975,630201,109873488,174.714876,174.730302,-0.015425,0.011131,-0.026556,-1,1000.0
6,2017-08-29,173.1854,173.2814,169.7774,170.9294,1632822,278939488,173.870161,174.054331,-0.184170,-0.038295,-0.145876,0,1000.0
7,2017-08-30,172.3214,172.8974,171.2654,171.3614,796773,136845984,173.346621,173.620433,-0.273812,-0.094894,-0.178918,0,1000.0
8,2017-08-31,172.5614,173.5694,171.9854,172.5614,1131043,195346992,173.191276,173.463461,-0.272185,-0.135849,-0.136336,0,1000.0
9,2017-09-01,173.0894,174.5775,172.6094,173.7134,935310,162351616,173.290218,173.497950,-0.207732,-0.151955,-0.055777,0,1000.0
